In [1]:
!pip install pretty_midi
!pip install xlstm
!pip install torch
!pip install note-seq
!pip install torch torchvision torchaudio
!pip install pretty_midi mido


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595886 sha256=cba12fb6c878a8e2346b07ff8715d83c0e232038c56c54902dde104d4c1e4ee1
  Stored in directory: /root/.cache/pip/wheels/f4/ad/93/a7042fe12668827574927ade9deec7f29aad2a1001b1501882
Successfully built pretty_midi
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.5 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [2]:
import torch
from xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig,
)

# 設定 xLSTM 配置
cfg = xLSTMBlockStackConfig(
    mlstm_block=mLSTMBlockConfig(
        mlstm=mLSTMLayerConfig(
            conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4
        )
    ),
    slstm_block=sLSTMBlockConfig(
        slstm=sLSTMLayerConfig(
            backend="cuda",  # 使用 GPU
            num_heads=4,
            conv1d_kernel_size=4,
            bias_init="powerlaw_blockdependent",
        ),
        feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),
    ),
    context_length=256,  # 每次處理的序列長度
    num_blocks=7,  # 堆疊區塊數量
    embedding_dim=128,  # 嵌入層維度
    slstm_at=[1],  # 指定哪些區塊使用 sLSTM
)

# 初始化 xLSTM 模型
xlstm_stack = xLSTMBlockStack(cfg)

# 創建隨機輸入數據並將其移動到 GPU
x = torch.randn(4, 256, 128).to("cuda")

# 將模型移動到 GPU
xlstm_stack = xlstm_stack.to("cuda")

# 前向傳播
y = xlstm_stack(x)

# 輸出結果
print("Output shape:", y.shape)

{'verbose': True, 'with_cuda': True, 'extra_ldflags': ['-L/usr/local/cuda/lib', '-lcublas'], 'extra_cflags': ['-DSLSTM_HIDDEN_SIZE=128', '-DSLSTM_BATCH_SIZE=8', '-DSLSTM_NUM_HEADS=4', '-DSLSTM_NUM_STATES=4', '-DSLSTM_DTYPE_B=float', '-DSLSTM_DTYPE_R=__nv_bfloat16', '-DSLSTM_DTYPE_W=__nv_bfloat16', '-DSLSTM_DTYPE_G=__nv_bfloat16', '-DSLSTM_DTYPE_S=__nv_bfloat16', '-DSLSTM_DTYPE_A=float', '-DSLSTM_NUM_GATES=4', '-DSLSTM_SIMPLE_AGG=true', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL_VALID=false', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL=0.0', '-DSLSTM_FORWARD_CLIPVAL_VALID=false', '-DSLSTM_FORWARD_CLIPVAL=0.0', '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_BFLOAT16_OPERATORS__', '-U__CUDA_NO_BFLOAT16_CONVERSIONS__', '-U__CUDA_NO_BFLOAT162_OPERATORS__', '-U__CUDA_NO_BFLOAT162_CONVERSIONS__'], 'extra_cuda_cflags': ['-Xptxas="-v"', '-gencode', 'arch=compute_80,code=compute_80', '-res-usage', '--use_fast_math', '-O3', '-Xptxas -O3', '--extra-device-vectorization', '-DSLSTM_

W1114 06:04:01.777000 606 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1114 06:04:01.777000 606 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
/usr/local/lib/python3.12/dist-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/usr/local/lib/python3.12/dist-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(


Output shape: torch.Size([4, 256, 128])


In [3]:
import gradio as gr
import torch
import torch.nn.functional as F
import json
import os
import pretty_midi
import mido
from typing import List

# -----------------------------
# Config / Paths
# -----------------------------
STYLE_MIDI_MAP = {
    "country": "midi_styles/country.mid",
    "hiphop": "midi_styles/hiphop.mid",
    "jazz": "midi_styles/jazz.mid",
    "rock": "midi_styles/rock.mid",
}

WORD_TO_ID_PATH = "word_to_id.json"
ID_TO_WORD_PATH = "id_to_word.json"
CHECKPOINT_PATH = "xlstm_best_model.pt"

# -----------------------------
# Load vocab
# -----------------------------
def load_vocab(word_to_id_path=WORD_TO_ID_PATH, id_to_word_path=ID_TO_WORD_PATH):
    with open(word_to_id_path, "r") as f:
        word_to_id = json.load(f)
    with open(id_to_word_path, "r") as f:
        raw = json.load(f)
        id_to_word = {int(k): v for k, v in raw.items()}
    return word_to_id, id_to_word

word_to_id, id_to_word = load_vocab()

device = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------
# Model loading
# -----------------------------
model = None
embedding = None
classifier = None

try:
    from xlstm import xLSTMBlockStack

    global cfg  # cfg must already exist in your environment

    if "cfg" in globals():
        model = xLSTMBlockStack(cfg).to(device)
        embedding = torch.nn.Embedding(303, cfg.embedding_dim).to(device)
        classifier = torch.nn.Linear(cfg.embedding_dim, 303).to(device)

        if os.path.exists(CHECKPOINT_PATH):
            ckpt = torch.load(CHECKPOINT_PATH, map_location=device)
            model.load_state_dict(ckpt.get("model_state_dict", {}))
            embedding.load_state_dict(ckpt.get("embedding_state_dict", {}))
            classifier.load_state_dict(ckpt.get("classifier_state_dict", {}))
except Exception as e:
    print("Model init failed:", e)


# -----------------------------
# Generation
# -----------------------------
def generate_music(start_tokens, model, embedding, classifier,
                   word_to_id, id_to_word, max_length=1024, device="cuda"):

    if model is None or embedding is None or classifier is None:
        raise RuntimeError("Model / embedding / classifier not initialized. Make sure cfg and checkpoint are loaded.")

    model.eval()
    generated = [word_to_id[t] for t in start_tokens]

    for _ in range(max_length):
        input_ids = torch.tensor(
            generated[-cfg.context_length:], dtype=torch.long
        ).unsqueeze(0).to(device)
        x_embed = embedding(input_ids)

        with torch.no_grad():
            output = model(x_embed)
            logits = classifier(output)

        prob = torch.softmax(logits[0, -1], dim=-1)
        next_id = torch.multinomial(prob, 1).item()
        generated.append(next_id)

        if id_to_word[next_id].startswith("[END]"):
            break

    return [id_to_word[i] for i in generated]


# -----------------------------
# Token → MIDI
# -----------------------------
def tokens_to_midi(tokens, bpm=120):
    midi = pretty_midi.PrettyMIDI()
    inst = pretty_midi.Instrument(program=0)

    beat = 60.0 / bpm
    grid = beat / 4  # resolution
    time = 0

    i = 0
    while i < len(tokens):
        t = tokens[i]

        if t.startswith("DELTA_"):
            delta = int(t.split("_")[1])
            time += delta * grid
            i += 1

        elif t.startswith("PITCH_") and i + 1 < len(tokens) and tokens[i + 1].startswith("VEL_"):
            pitch = int(tokens[i].split("_")[1])
            vel = min(int(tokens[i + 1].split("_")[1]) * 8, 127)
            inst.notes.append(
                pretty_midi.Note(velocity=vel, pitch=pitch, start=time, end=time + grid)
            )
            i += 2
        else:
            i += 1

    midi.instruments.append(inst)
    return midi


# -----------------------------
# MIDI ↔ Pianoroll Helpers
# -----------------------------
def midi_to_pianoroll(midi_path, fs=8):
    """
    讀取 MIDI 檔並轉成 piano-roll: [128, T] tensor
    """
    pm = pretty_midi.PrettyMIDI(midi_path)
    roll = torch.tensor(pm.get_piano_roll(fs=fs), dtype=torch.float32)  # [128, T]
    return roll


def pianoroll_to_midi(piano_roll, fs=8, program=0):
    """
    將 piano-roll: [128, T] 轉回單一 instrument 的 MIDI
    """
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    piano_roll = piano_roll > 0  # bool mask
    notes, frames = piano_roll.shape

    for pitch in range(notes):
        row = piano_roll[pitch]
        on = False
        start = 0

        for i, val in enumerate(row):
            if val and not on:
                on = True
                start = i
            elif not val and on:
                on = False
                end = i
                instrument.notes.append(
                    pretty_midi.Note(
                        velocity=100,
                        pitch=pitch,
                        start=start / fs,
                        end=end / fs,
                    )
                )

        # 若最後還在發聲，收尾
        if on:
            instrument.notes.append(
                pretty_midi.Note(
                    velocity=100,
                    pitch=pitch,
                    start=start / fs,
                    end=frames / fs,
                )
            )

    midi.instruments.append(instrument)
    return midi


# -----------------------------
# Mix two MIDI (Pianoroll mix)
# -----------------------------
def mix_midi(user_midi_path, style, alpha=0.5, fs=8):
    """
    使用 piano-roll 做混音：
    alpha 越接近 1 -> 越偏向 style MIDI
    alpha 越接近 0 -> 越偏向生成的 MIDI
    """
    if style not in STYLE_MIDI_MAP:
        return user_midi_path

    style_path = STYLE_MIDI_MAP[style]
    if not os.path.exists(style_path):
        return user_midi_path

    # 轉成 piano-roll
    pr1 = midi_to_pianoroll(user_midi_path, fs=fs)   # 生成的
    pr2 = midi_to_pianoroll(style_path, fs=fs)       # style MIDI

    # 對齊長度
    T = max(pr1.shape[1], pr2.shape[1])
    if pr1.shape[1] < T:
        pr1 = F.pad(pr1, (0, T - pr1.shape[1]))
    if pr2.shape[1] < T:
        pr2 = F.pad(pr2, (0, T - pr2.shape[1]))

    # 混音：alpha 越大越偏向 style
    mixed_roll = alpha * pr2 + (1.0 - alpha) * pr1

    # 二值化：是否發聲
    mixed_roll = (mixed_roll > 0.5).float()

    # 轉回 MIDI
    midi = pianoroll_to_midi(mixed_roll, fs=fs, program=0)
    out_path = f"mixed_{style}.mid"
    midi.write(out_path)
    return out_path


# -----------------------------
# Gradio Call Function
# -----------------------------
def run(start_genre, delta, pitch, vel, alpha, do_mix, mix_style):

    start_tokens = [
        f"[GENRE_{start_genre.upper()}]",
        f"DELTA_{int(delta)}",
        f"PITCH_{int(pitch)}",
        f"VEL_{int(vel)}"
    ]

    tokens = generate_music(start_tokens, model, embedding, classifier,
                            word_to_id, id_to_word, device=device)

    midi = tokens_to_midi(tokens)
    out = "generated.mid"
    midi.write(out)

    if do_mix:
        # 把 UI 的 alpha 丟進去
        return mix_midi(out, mix_style, alpha=float(alpha))

    return out


# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks(title="MIDI Generator Minimal") as demo:
    gr.Markdown("## 🎵 Minimal MIDI Generator with Style Mix")

    genre = gr.Dropdown(["CLASSICAL", "COUNTRY", "HIPHOP", "JAZZ", "ROCK"],
                        label="Genre")
    delta = gr.Number(value=5, label="DELTA")
    pitch = gr.Number(value=95, label="PITCH")
    vel = gr.Number(value=10, label="VEL")

    # 新增 alpha Slider：0=純模型生成，1=純 style MIDI
    alpha = gr.Slider(
        minimum=0.0,
        maximum=1.0,
        value=0.5,
        step=0.05,
        label="Mix Alpha (0 = Pure Generated, 1 = Pure Style)"
    )

    do_mix = gr.Checkbox(label="Mix with style MIDI", value=False)
    mix_style = gr.Dropdown(
        list(STYLE_MIDI_MAP.keys()),
        label="Mix Style",
        value="jazz"
    )

    btn = gr.Button("Generate MIDI")
    out_file = gr.File(label="Output MIDI")

    # 記得把 alpha 放進 inputs
    btn.click(run, [genre, delta, pitch, vel, alpha, do_mix, mix_style], out_file)

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/usr/local/lib/python3.12/dist-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e60054376a8922b4d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
